# PROJET HADOOP/SPARK - MS-SIO-2019 
## CONSOMMATEUR FLUX KAFKA
---
### P.Hamy, N.Leclercq, L.Poncet
---

#### Ce notebook Jupyter implémente la partie _consommateur_ du flux Kafka dans lequel sont injectées les données issues de l'API Transilien.  

Import des packages Python requis

In [1]:
import os
import json
import time
import logging
from pyspark.sql import SparkSession
import pyspark.sql.types as st
import pyspark.sql.functions as sf
from pyspark.sql.window import Window as spark_window

Imports de nos outils locaux:
- NotebookCellContent permet d'attacher la "cellule courante" du notebook Jupyter à l'objet qui hérite de cette classe. Le principal intérêt est de router le logging asynchrone vers cette cellule et ce quelle que soit la cellule active (i.e. quelle que soit la cellule dans laquelle l'utilisateur travaille). Le lien entre l'objet python et la cellule du notebook s'effectue à l'instanciation de l'objet - c'est pourquoi la cellule cible est celle dans laquele l'objet est crée.

In [ ]:
from tools.logging import NotebookCellContent

Configuration de base du logging Python.

In [ ]:
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.ERROR, datefmt='%H:%M:%S')

Changement du logging level de la couche Java (*) afin d'éliminer le bruit généré dans la console par un [_warning_](https://stackoverflow.com/questions/39351690/got-interruptedexception-while-executing-word-count-mapreduce-job) récurrent.

(*) Spark est implémenté en Java. PySpark est une interface Python qui s'appuie sur [py4j](https://www.py4j.org/) afin d'autoriser l'accès aux objets Java instanciés dans la JVM.

In [ ]:
log4j = sc._jvm.org.apache.log4j
log4j.LogManager.getRootLogger().setLevel(log4j.Level.ERROR)

---
### PARTIE I - QUESTION 1.1 : calcul et publication du temps d'attente moyen par station
---

Cette section donne le détail de construction du flux de données relatif à la dernière heure écoulée - tranche horaire à laquelle les métriques demandées s'appliquent. Le code est segmenté afin d'en faliciter le commentaire. Il sera repris plus loin et encapsulé dans une classe dédiée. 

**Note: la partie II du projet étant implémentée sur le même schéma, on fait le choix de ne détailler aussi finement que cette première partie du projet.**

#### Création de la session Spark

In [ ]:
kafka_session = SparkSession \
                .builder \
                .appName("MS-SIO-HADOOP-PROJECT-KAFKA-STREAM") \
                .config("spark.sql.shuffle.partitions", 4) \
                .getOrCreate()

#### Création du flux Kafka
On crée un [structured spark stream](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html) associé à une source Kafka. Il s'agit de spécifier la source des messages via l'adresse du serveur Kafka et le nom du topic auquel on souhaite s'abonner. 

On chosit de potionner l'option __startingOffsets__ à la valeur __earliest__ de façon à obtenir toutes les données présentes (i.e. stockées) dans le topic à chaque redémarrage de notre application. Nous avosn toutefois pris soin de limiter le durée de rétention des messages à 8 heures - un réglage propre à chaque topic effectuer comme suit:
```
cd /usr/hdp/current/kafka-broker
./bin/kafka-topics.sh --zookeeper localhost:2181 --alter --topic transilien-02 --config retention.ms=28800000
```

In [ ]:
kafka_stream = kafka_session \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "sandbox-hdp.hortonworks.com:6667") \
    .option("subscribe", "transilien-02") \
    .option("startingOffsets", "earliest") \
    .option("kafkaConsumer.pollTimeoutMs", 512) \
    .load()

#### Schéma de désérialisation des messages  
Au niveau du _producteur_, les messages injectés dans le flux Kafka sont sérialisés et encodés au format JSON (encodage binaire). Le connecteur Kafka les delivre sous la forme d'un dataframe dont le schéma est le suivant :
```
kafka_stream.printSchema()
root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)
 ```

Le champ _value_ du dataframe contient les données utiles du message - données qu'il est nécessaire de décoder. Le schéma de désérialisation **json_schema** indique à la fonction Spark **from_json** comment extraire l'information contenue dans le message :

In [ ]:
json_schema = st.StructType(
    [
        st.StructField("station", st.IntegerType(), True),
        st.StructField("train", st.StringType(), True),
        st.StructField("timestamp", st.TimestampType(), True),
        st.StructField("mode", st.StringType(), True),
        st.StructField("mission", st.StringType(), True),
        st.StructField("terminus", st.IntegerType(), True)
    ]
)

A travers, la variable **json_options**, on précise également le format du champ _timestamp_ afin que les champs du type _TimestampType_ soient correctement interprétées.

In [ ]:
json_options = {"timestampFormat": "yyyy-MM-dd'T'HH:mm:ss.sss'Z'"}

#### Séquence de calcul du temps d'attente moyen par station : étape-01 
Le traitement du flux Kafka commence donc par une phase désérialisation/reformatage des messages.

In [ ]:
df = kafka_stream \
    .select(sf.from_json(sf.col("value").cast("string"), json_schema, json_options).alias("departure")) \
    .select("departure.*")

A l'issue de cette opération le dataframe _df_ a le schéma suivant :
```
df.printSchema()
root
 |-- station: integer (nullable = true)
 |-- train: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- mode: string (nullable = true)
 |-- mission: string (nullable = true)
 |-- terminus: integer (nullable = true)
```
#### Séquence de calcul du temps d'attente moyen par station : étape-02
Spécification de la watermark du stream Spark. Nous n'acceptons pas les messages ayant plus d'une minute de retard par rapport à la fin de la fenêtre temorelle à laquelle ils appartiennent. Il s'agit d'un choix arbitraire qui n'a que peu d'intérêt dans notre projet. Il est toutefois nécessaire de spécifier cette valeur car l'implémentation sous-jacente doit borner l'accumulation des données en RAM (contrainte liée au mécanisme de fenêtre glissante). [La documentation de Spark explique clairement le concept de _Stateful Stream Processing in Structured Streaming_](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html).

In [ ]:
df = df.withWatermark("timestamp", "1 minutes")

#### Séquence de calcul du temps d'attente moyen par station : étape-03
Un train apparaitra dans les réponses aux requêtes de l'API SNCF tant que son heure de départ n'appartient pas au passé (voire un 'certain' temps après son départ). On supprime donc les doublons associés aux couples (train, heure de départ). Inutile d'ajouter la station à la contrainte d'exclusion car l'idenfiant d'un train est unique.

In [ ]:
df = df.dropDuplicates(["train", "timestamp"])

#### Séquence de calcul du temps d'attente moyen par station : étape-04
Définition de la fênêtre (temporelle) dans laquelle les calculs sont aggrégés. Il s'agit ici de calculer le temps d'attente moyen par station **sur la dernière heure écoulée**. Notre fenêtre a donc une largeur temporelle de 60 minutes (_window length_). On choisit de suivre cette moyenne par pas de 2 minutes (_sliding interval_). Dans la mesure où le calcul est demandé par station, le _groupBy_ s'applique au champ _station_ du dataframe.

In [ ]:
df = df.groupBy("station", sf.window("timestamp", "60 minutes", "2 minutes"))

#### Séquence de calcul du temps d'attente moyen par station : étape-05
**Pour chaque station, on définit le temps moyen d'attente sur une période de P minutes comme le rapport de P vs le nombre de trains au départ de cette station sur la période P**. 

Ici P = 60 minutes. 

On crée ainsi une _aggrégation_ qui contiendra, pour chaque station et pour chaque fenêtre d'une heure :
- une colonne _nt_ indiquant le nombre de trains sur la période
- une colonne _awt_ donnant le temps d'attente moyen recherché  (_awt = Average Waiting Time_). 

La colonne _nt_ est injectée dans le dataframe à titre indicatif (visualisation dans la console, validation du calcul).

In [ ]:
df = df.agg(sf.count("train").alias("nt"), sf.format_number(60./ sf.count("train"), 2).cast("double").alias("awt"))

#### Séquence de calcul du temps d'attente moyen par station : étape-06
Identification et selection de la fenêtre temporelle associée à la dernière heure écoulée. 

Il s'agit de selectionner, parmis les N fenêtres temporelles produites par Spark, celle qui correspond à la dernière heure écoulée. On utilise ici la fonction **current_timestamp** de Spark afin de rendre la sélection dynamique (i.e. glissante). Le calcul est effectué dans l'unité de **unix_timestamp** (i.e. la seconde) - beaucoup plus facile à manipuler dans ce contexte. 

Dans l'idée de pourvoir visualiser (si besoin) les valeurs mises en jeu dans le calcul, on choisit de créér les colonnes associées:
- oha = one hour ago = now - 62 minutes = now - (window length + sliding interval) 
- now = now - 1 minutes = now - (sliding interval / 2.) => **valeur ajustée pour n'obtenir qu'une seule fenêtre**
- wstart = window.start = borne inférieure de la fenêtre temporelle 
- wend = window.wend = borne supérieure de la fenêtre temporelle

La clause _where_ permet de selectionner la fenêtre associée à la dernière heure.  

In [ ]:
df = df \
    .withColumn("oha", sf.unix_timestamp(sf.current_timestamp()) - int((60 + 2) * 60)) \
    .withColumn("now", sf.unix_timestamp(sf.current_timestamp()) - int(60 * 2) / 2.) \
    .withColumn("wstart", sf.unix_timestamp("window.start")) \
    .withColumn("wend", sf.unix_timestamp("window.end")) \
    .where((sf.col("oha") <= sf.col("wstart")) & (sf.col("wend") <= sf.col("now")))

Note: à ce stade, _df_ constitue 'l'état' de référence de notre stream Spark. C'est à partir de cet état que l'on produit les résultats, métriques, indicateurs, ... relatifs à la dernière haure de données.

#### Validation des étapes 01 à 06

On lance un 'writeStream' vers la console afin de visualiser les données produites par le stream.

In [ ]:
query = df \
    .select("station", "window", "nt", "awt") \
    .writeStream \
    .format("console") \
    .option("truncate", False) \
    .outputMode("complete") \
    .start()

````
+--------+------------------------------------------+---+----+
|station |window                                    |nt |awt |
+--------+------------------------------------------+---+----+
|87382473|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|14 |4.29|
|87386425|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|14 |4.29|
|87382259|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|19 |3.16|
|87382457|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|18 |3.33|
|87382440|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|20 |3.0 |
|87382333|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|14 |4.29|
|87382887|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|12 |5.0 |
|87334482|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|30 |2.0 |
|87381137|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|48 |1.25|
|87386318|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|17 |3.53|
|87382374|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|12 |5.0 |
|87382499|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|27 |2.22|
|87382655|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|5  |12.0|
|87382382|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|25 |2.4 |
|87381905|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|31 |1.94|
|87384008|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|16 |3.75|
|87386003|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|19 |3.16|
|87386300|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|24 |2.5 |
|87381129|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|38 |1.58|
|87386409|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|24 |2.5 |
+--------+------------------------------------------+---+----+
```

Arrêt de la query

In [ ]:
query.stop()

In [ ]:
kafka_session.streams.active

---
### PARTIE I - QUESTIONS 1.2 à 1.6
---
- Calcul du temps moyen d’attente sur la ligne station par station sur la dernière heure
- Calcul du temps moyen d’attente globale sur la ligne sur la dernière heure
- Trier les stations par temps d’attente moyen sur la dernière heure
- Trouver la station avec le temps d’attente le plus élevée sur la dernière heure
- Trouver la station avec le temps d’attente le moins élevée sur la dernière heure
- Construire un tableau de bord dans Tableau Software sur la base de ces indicateurs

Les calculs demandés nécessiteraient une seconde opération aggrégation sur le stream (incarné dans ce qui précède par le dataframe _df_). Or, en l'état actuel de Spark (2.4), [il n'est pas possible d'enchainer plusieurs opération d'aggrégation sur un même stream](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#unsupported-operations). Il nous faut donc trouver une solution de contournement.

L'idée retenue est d'effectuer les calculs sur chaque _batch_ dans le contexte d'un callback du type _foreachBatch_. Cette approche fonctionne ici car chaque _batch_ contient l'intégralité des données sur lesquelles les calculs doivent être réalisés pour produire les résultats attendus - i.e. les données d'attente moyenne par station. 

Retenons que l'idée d'effectuer les calculs dans le contexte d'un callback du type _foreachBatch_ nous est venue après bien des recherches sur le net ! 

Ces résulats seront enregistrés en local sous la forme de _vues temporaires_. Ils sont rendus accéssibles depuis Tableau par l'intermédaire de notre propre serveur **_thrift_** - i.e. lancé en local et configuré de manière spécifique (i.e. en mode _single session_). 

Les calculs (Q1.2 à Q.1.5) sont regroupés dans un callback du type _foreachBatch_ dont les appels sont déclenchés par une _StreamingQuery_.

**La classe _TransilienStreamProcessor_ implémente les fonctionnalités de la partie I du projet à travers les fonctions setup_last_hour_awt_stream et computeAwtMetricsAndSaveAsTempViews**.

---
### PARTIE II - Calcul de la progression des trains et localisation géographique.
---

Nous proposons une solution globale qui répond à l'ensemble des demandes et implémente la totalité des '_bonus_':

Demandes de base:

- calculer la durée du parcours d’un train entre les stations A et B
- calculer en continu (à partir de l’heure courante) le % du trajet réalisé par le train 
- afficher la barre de progression du trajet dans Tableau Software (avec rafraichissement automatique)

Bonus :
- afficher la progression de l’ensemble des trains de la ligne
- géolocaliser de l’ensemble des trains de la ligne
- publier le code sous gitlab (ou github)

Dans son implémetation l'approche est similaire à celle des questions de la partie I et utilise également un callback du type _foreachBatch_. 

**La classe _TransilienStreamProcessor_ implémente les fonctionnalités de la partie II du projet à travers les fonctions setup_trains_progression_stream et computeTrainsProgressionAndSaveAsTempView**.

----
### Implementation de la solution proposée pour les parties I & II du projet
---

Import des packages Python requis

In [1]:
import os
import json
import time
import logging
import numpy as np
from scipy.interpolate import interp1d
from pyspark.sql import SparkSession
import pyspark.sql.types as st
import pyspark.sql.functions as sf
from pyspark.sql.window import Window as spark_window
from py4j.java_gateway import java_import
from tools.task import Task
from tools.logging import NotebookCellContent

Configuration de base du logging Python.

In [2]:
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.ERROR, datefmt='%H:%M:%S')

Changement du logging level de la couche Java (*) afin d'éliminer le bruit généré dans la console par un [_warning_](https://stackoverflow.com/questions/39351690/got-interruptedexception-while-executing-word-count-mapreduce-job) récurrent.

(*) Spark est implémenté en Java. PySpark est une interface Python qui s'appuie sur [py4j](https://www.py4j.org/) afin d'autoriser l'accès aux objets Java instanciés dans la JVM.

In [3]:
log4j = sc._jvm.org.apache.log4j
log4j.LogManager.getRootLogger().setLevel(log4j.Level.ERROR)

Les trois fonctions suivantes sont liées aux _UDF_ Spark (_User Defined Fonctions_) que nous mettons en oeuvre dans le calcul de la position 'temps réel' des trains. 

Nous proposons en fait deux modes de calcul: 
- une première évaluation de la position de chaque train est calculée sur la base d'une interpolation linéaire de la latitude et de la longitude entre les gares de départ et d'arrrivée. L'inconvénient de cette approche est de produire des positions incohérentes (hors rails) dans le cas des 'trains directs'. 

- le second mode de calcul de la positioin des trains est basé sur un "modèle géolocalisée" de la ligne. Ce modèle, construit sur la base des données SNCF et stocké dans le fichier '_scnf-paths-line-l.json_', décrit la ligne de station en station. Quelles que soient les stations _sA_ et _sB_ de la ligne, le parcours _sA_ -> _sB_ est stocké sous la forme d'un vecteur de _Np_ points géolocalisés qui suivent le tracé des rails entre les gares _sA_ et _sB_. Ces données permettent d'interpoler la position du train de manière beaucoup plus précise. On notera que l'interpolation entre deux points du parcours est - en fonction de _Np_ - du type _cubic spline_ ou _linear_ et qu'elle repose sur les fonctionnalirés du package _scipy_. Enfin, on retiendra que ce calcul est delégué à des UDF Spark et qu'il peut être déactivé à la demande (afin d'en constater visuellement l'effet). 

**A propos des fonctions UDF de Spark :**
Une contrainte technique liée à Spark ne permet pas d'associer les fonctions membres d'une instance de classe à une UDF Spark. Pour faire court, la référence _self_ - qui désigne une instance particulière d'une classe - [ne peut être _broadcastée_ ](https://stackoverflow.com/questions/31396323/spark-error-it-appears-that-you-are-attempting-to-reference-sparkcontext-from-a/31600775) - i.e. propogagée du _driver_ vers les _workers_. [Cet article](https://jaceklaskowski.gitbooks.io/mastering-apache-spark/spark-broadcast.html) explique en détail ce qu'est le mécanique de broadcast _driver_ -> _worker_ des variables dans Spark. Nous sommes donc contraints d'utiliser des fonctions indépendantes. D'où la présence des trois fonctions suivantes. Idéalement, on aurait souhaité les encapsuler dans notre classe d'implémentation du projet: **TransilienStreamProcessor**.

In [4]:
# --------------------------------------------------------------------
# load trains paths data (geolocalized paths from station to station)
with open("./scnf-paths-line-l.json", "r", encoding="utf-8") as f:
    g_trains_paths = json.load(f)

# --------------------------------------------------------------------
def accurate_latitude(from_station, to_station, progression):  
    # compute accurate latitude
    return accurate_train_position('lat', from_station, to_station, progression)

# --------------------------------------------------------------------
def accurate_longitude(from_station, to_station, progression):  
    # compute accurate longitude
    return accurate_train_position('lon', from_station, to_station, progression)  

# --------------------------------------------------------------------
def accurate_train_position(geo_component, from_station, to_station, train_progression):
    # compute accurate trains positions
    # the interpolation is done along the paths stored into <g_trains_paths>
    path_from_st_to_st = g_trains_paths[f"{from_station}-{to_station}"]
    # special case for 'standby trains' (trains waiting for departure)
    if from_station == to_station:
        if geo_component == 'lat':
            geo_pos = float(path_from_st_to_st["geoPoints"][0]["latitude"])
        else:
            geo_pos = float(path_from_st_to_st["geoPoints"][0]["longitude"])
    else:    
        # the scipy cubic spline interpolator doesn't like short interpolation domains 
        # use cubic spline when the number of points between from_station & to_station is above 4
        interpolator = 'linear' if path_from_st_to_st["number_step"] <= 4 else 'cubic'
        # prepare data for interpolator: progression axis in % 
        x = np.linspace(0., 100., num=path_from_st_to_st["number_step"], endpoint=True)
        # interpolation of the requested pos. component (latitude or longitude)
        if geo_component == 'lat':
             # prepare data for interpolator: latitude axis
            lat_y = [float(point["latitude"]) for point in path_from_st_to_st["geoPoints"]]
            # get latitude for the cuurent 'train_progression'
            geo_pos = float(interp1d(x, lat_y, kind=interpolator)(train_progression))
        else:
            # prepare data for interpolator: longitude axis 
            lon_y = [float(point["longitude"]) for point in path_from_st_to_st["geoPoints"]]
            # get longitude for the cuurent 'train_progression'
            geo_pos = float(interp1d(x, lon_y, kind=interpolator)(train_progression))
    # return current latitude and longitude values
    return geo_pos

**TransilienStreamProcessor**: notre classe de traitement du stream API Transilien

In [5]:
class TransilienStreamProcessor(NotebookCellContent):
    
    # unique TransilienStreamProcessor instance
    singleton = None
    
    # spark user defined fonction: accurate latitude computation
    alt_udf = sf.udf(accurate_latitude, st.FloatType())
    
    # spark user defined fonction: accurate longitude computation
    alg_udf = sf.udf(accurate_longitude, st.FloatType())
    
    # -------------------------------------------------------------------------------
    def __init__(self, config):
    # -------------------------------------------------------------------------------
        # init mother class (NotebookCellContent details above)
        NotebookCellContent.__init__(self, "TransilienStreamProcessor")
            
        # store the configuration 
        self.config = config
        
        # setup logging: timestamp of the last cell clearing (avoid accumulating log in the notebook cell)
        self.last_clear_outputs_ts = time.time()
        # setup logging: show/display train progression table (computation result)
        self.show_trprg_table = self.config.get('show_trprg_table', False)
        # setup logging: temp logging level (we want to see the init messages whatever is the logging level)
        self.set_logging_level(logging.DEBUG)
        
        # release any existing instance
        if TransilienStreamProcessor.singleton is not None:
            self.warning("TSP:releasing existing instance...")
            try:
                TransilienStreamProcessor.singleton.stop()
                del(TransilienStreamProcessor.singleton)
            except Exception as e:
                print(e)
            self.warning("TSP:`-> done!")
            
        # self is the unique TransilienStreamProcessor instance
        TransilienStreamProcessor.singleton = self
  
        # enable 'accurate trains position' computation
        self.accurate_trains_position_enabled = True
        
        self.debug("TSP:initializing...")
        
        # kafka oriented spark session (i.e. configured to process incoming Kafka messages)
        # we notably specify the thrift server mode and port
        self.debug(f"TSP:creating kafka oriented spark session")
        self.kafka_session = SparkSession \
            .builder \
            .master("yarn") \
            .appName("MS-SIO-HADOOP-PROJECT-STREAM") \
            .config("spark.sql.shuffle.partitions", self.config['spark_sql_shuffle_partitions']) \
            .config('spark.sql.hive.thriftServer.singleSession', True) \
            .config('hive.server2.thrift.port', self.config['hive_thrift_server_port']) \
            .enableHiveSupport() \
            .getOrCreate()
        self.debug("`-> done!")
                
        # average waiting time on the last hour of data (awt): kafka stream setup
        self.debug(f"TSP:initializing 'last hour awt' stream")
        self.last_hour_awt_stream = self.__setup_last_hour_awt_stream()
        self.debug("`-> done!")
        
        # real time trains progression: kafka stream setup
        self.debug(f"TSP:initializing 'trains progression' stream")
        self.trains_progression_stream = self.__setup_trains_progression_stream()
        self.debug("`-> done!")
        
        # start our own thrift server
        # this will allow to expose the temp. views and make them reachable from Tableau Software   
        self.__start_thrift_server()
            
        # create a temp. view for the transilien stations data (label, geo.loc., ...)
        # this will allow to expose this data and make them reachable from Tableau Software
        self.stations_data = self.__create_stations_view()
        
        # average waiting time on the last hour of data (awt): computation streaming query
        # acts as a trigger for computeAwtMetricsAndSaveAsTempViews (forEachBatch callback)
        self.lhawt_sink = None
        
        # average waiting time on the last hour of data (awt): console logging streaming query
        # this will allow to print batches into the console  
        self.lhawt_console_sink = None
        
        # real time trains progression (trprg): computation streaming query
        # acts as a trigger for computeTrainsProgressionAndSaveAsTempView (forEachBatch callback)
        self.trprg_sink = None

        # start the streaming queries?
        if self.config['auto_start']:
            self.start()
        
        self.debug(f"initialization done!")
        
        # set actual logging level (the one specified by the configuration)
        self.set_logging_level(logging.DEBUG if self.config['verbose'] else logging.ERROR)
        
    # -------------------------------------------------------------------------------
    def __start_thrift_server(self):
    # -------------------------------------------------------------------------------
        # start our own thrift server
        # port and mode were specified at 'kafka_session' instanciation
        # ---------------------------------------------------------------------------
        # note: there is now way to stop the server once started! there's also no way
        # note: to check whether or not the thrift server is already running! we 
        # note: consequently have to restart our python kernel  each time we want to 
        # note: change something into the code - that's real annoying! the best 
        # note: workaround we have is to start the server is it's not already running!
        # ---------------------------------------------------------------------------
        def __is_thrift_server_running(port):
            import socket
            thrift_server_running = False
            try:
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                s.connect(("localhost", port))
                s.close()
                thrift_server_running = True
            except ConnectionRefusedError:
                pass
            return thrift_server_running
        if __is_thrift_server_running(self.config['hive_thrift_server_port']):
            wm = f"TSP:thrift server already running on port {self.config['hive_thrift_server_port']}"
            self.warning(wm) 
            return
        try:
            self.debug(f"TSP:starting thrift server on port {self.config['hive_thrift_server_port']}") 
            #sc.setLogLevel('INFO')
            java_import(sc._gateway.jvm,"")
            sc._gateway.jvm.org.apache.spark.sql \
                       .hive.thriftserver.HiveThriftServer2 \
                       .startWithContext(spark._jwrapped)
            #sc.setLogLevel('ERROR')       
            self.debug(f"TSP:thrift server successfully started") 
        except Exception as e:
            self.error(e)
                       
    # -------------------------------------------------------------------------------
    def __create_stations_view(self):
    # -------------------------------------------------------------------------------
        # read then save stations data as a tmep view so that we can retrieve it from
        # Tableau Software (or any client having the ability to talk to our thrift server)         
        df = self.kafka_session \
            .read \
            .format("csv") \
            .option("sep", ",") \
            .option("inferSchema", "true") \
            .option("header", "true") \
            .load("file:/root/ms-sio-hdp/api-transilien/transilien_line_l_stations_by_code.csv")
        df.createOrReplaceTempView("stations_data")
        return df
                       
    # -------------------------------------------------------------------------------
    def enable_accurate_trains_position(self):
    # -------------------------------------------------------------------------------
         # enable 'accurate trains position' feature
         self.accurate_trains_position_enabled = True
    
    # -------------------------------------------------------------------------------
    def disable_accurate_trains_position(self):
    # -------------------------------------------------------------------------------
         # disable 'accurate trains position' feature
         self.accurate_trains_position_enabled = False
                
    # -------------------------------------------------------------------------------
    def __setup_last_hour_awt_stream(self):
    # -------------------------------------------------------------------------------
        # setup stream for the 'average waiting time on the last hour of data' 
        # --------------------------------------------------------------------
        # processing sequence comments:
        # 1 - create the kafka stream
        # 2 - extract data from kafka messages (i.e. deserialization) 
        # 3 - set watermark (using kafka_lhawt_stream_watermark config parameter)
        # 4 - drop (train, departure-time) duplicates         
        # 5 - setup sliding window (using config parameters)
        # 6 - count trains in each window & compute average waiting time 
        # 7 - select 'last hour window'
        # 8 - drop temp. columns
        wm = float(self.config['kafka_lhawt_stream_watermark'])
        wl = float(self.config['kafka_lhawt_stream_window_length'])
        si = float(self.config['kafka_lhawt_stream_sliding_interval'])
        oha_offset = int((wl + si) * 60.)
        now_offset = int(60. * si / 2.)
        return self.kafka_session \
            .readStream \
            .format("kafka") \
            .option("kafka.bootstrap.servers", self.config['kafka_broker']) \
            .option("subscribe", self.config['kafka_topic']) \
            .option("spark.streaming.kafka.consumer.poll.ms", 100) \
            .option("startingOffsets", "earliest") \
            .load() \
            .select(sf.from_json(sf.col("value").cast("string"), 
                                 self.config['json_schema'], 
                                 self.config['json_options']).alias("departure")) \
            .select("departure.*") \
            .withWatermark("timestamp", f"{int(wm)} minutes") \
            .dropDuplicates(["train", "timestamp"]) \
            .groupBy("station", sf.window("timestamp", f"{int(wl)} minutes", f"{int(si)} minutes")) \
            .agg(sf.count("train").alias("nt"), \
                 sf.format_number(wl / sf.count("train"), 1).cast("double").alias("awt")) \
            .withColumn("oha", sf.unix_timestamp(sf.current_timestamp()) - oha_offset) \
            .withColumn("now", sf.unix_timestamp(sf.current_timestamp()) - now_offset) \
            .withColumn("wstart", sf.unix_timestamp("window.start")) \
            .withColumn("wend", sf.unix_timestamp("window.end")) \
            .where((sf.col("oha") <= sf.col("wstart")) & (sf.col("wend") <= sf.col("now"))) \
            .drop("oha", "now", "wstart", "wend")
      
    # -------------------------------------------------------------------------------
    def __setup_trains_progression_stream(self):
    # -------------------------------------------------------------------------------
        # setup stream for trains progression 
        # -----------------------------------
        # processing sequence comments:
        # 1 - create the kafka stream
        # 2 - extract data from kafka messages (i.e. deserialization) 
        # 3 - drop (train, departure-time) duplicates
        # 4 - filter on departure-time mode ('R' only)
        # 5 - convert departure time (i.e. timestamp) to unix timestamp then drop initial column
        # 6 - filter on 'time window' (keep only trains which departure time is in now +/- half-time-window) 
        # 7 - execute a 'dummy' aggregation so that we can work in 'complete' mode (nice trick)
        # 8 - order by ("train", "departure", "station")
        time_window = config['kafka_trprg_time_window']            
        return self.kafka_session \
            .readStream \
            .format("kafka") \
            .option("kafka.bootstrap.servers", "sandbox-hdp.hortonworks.com:6667") \
            .option("subscribe", "transilien-02") \
            .option("spark.streaming.kafka.consumer.poll.ms", 100) \
            .option("startingOffsets", "earliest") \
            .load() \
            .select(sf.from_json(sf.col("value").cast("string"), \
                                 self.config['json_schema'], \
                                 self.config['json_options']).alias("departure")) \
            .select("departure.*") \
            .dropDuplicates(["train", "timestamp"]) \
            .filter("mode='R'") \
            .withColumn("departure", sf.unix_timestamp("timestamp")).drop("timestamp") \
            .where(sf.col("departure") \
            .between(sf.unix_timestamp(sf.current_timestamp()) - int(time_window/2.), \
                                       sf.unix_timestamp(sf.current_timestamp()) + int(time_window/2.))) \
            .groupBy("train", "station", "departure", "mode", "mission", "terminus") \
            .agg(sf.count("train").alias("tmp")).drop("tmp") \
            .orderBy("train", "departure", "station")
                       
    # -------------------------------------------------------------------------------
    def start(self):
    # -------------------------------------------------------------------------------
        # start streaming queries
        self.start_last_hour_awt_stream()
        self.start_trains_progress_stream()
     
    # -------------------------------------------------------------------------------
    def stop(self):
    # -------------------------------------------------------------------------------
        # stop streaming queries
        self.stop_last_hour_awt_stream()
        self.stop_trains_progress_stream()
                       
    # -------------------------------------------------------------------------------
    def start_last_hour_awt_stream(self):
    # -------------------------------------------------------------------------------
        # stop the streaming queries if already running
        self.stop_last_hour_awt_stream()            
        # processing time of the queries
        proc_time = f"{self.config.get('kafka_lhawt_processing_time', 5.)} seconds"     
        # last hour awt: create then start the (computation) streaming query
        # also make 'self.computeAwtMetricsAndSaveAsTempViews' the 'foreachBatch' callback
        self.debug(f"TSP:starting 'awt' sink (stream query)")
        self.lhawt_sink =  self.last_hour_awt_stream \
                            .writeStream \
                            .trigger(processingTime=proc_time) \
                            .foreachBatch(self.computeAwtMetricsAndSaveAsTempViews) \
                            .outputMode("complete") \
                            .start()
        self.debug(f"`-> done!")    
        if self.config.get('kafka_lhawt_console_sink_enabled', False):
            # last hour awt: create then start the (console) streaming query
            self.debug(f"TSP:starting 'awt' console stream (stream query)")
            self.lhawt_console_sink = self.last_hour_awt_stream \
                                .orderBy("awt") \
                                .writeStream \
                                .trigger(processingTime=proc_time) \
                                .outputMode("complete") \
                                .format("console") \
                                .option("truncate", False) \
                                .start() 
        self.debug(f"`-> done!")
                       
    # -------------------------------------------------------------------------------
    def start_trains_progress_stream(self):
    # -------------------------------------------------------------------------------
        # stop the streaming queries if already running
        self.stop_trains_progress_stream()
        # trains progression: create then start the hive sink (streaming query)
        # also make 'self.computeTrainsProgressionAndSaveAsTempView' the 'foreachBatch' callback
        self.debug(f"TSP:starting trains progression sink (stream query)")
        self.trprg_sink = self.trains_progression_stream \
                            .writeStream \
                            .foreachBatch(self.computeTrainsProgressionAndSaveAsTempView) \
                            .outputMode("complete") \
                            .start()
        self.debug(f"`-> done!")
        self.debug(f"TSP:streaming queries are running")
                       
    # -------------------------------------------------------------------------------
    def stop_last_hour_awt_stream(self):
    # -------------------------------------------------------------------------------
        # stop the streaming queries (best effort impl.)
        if self.lhawt_sink is  not None:
            try:
                self.debug(f"TSP:stopping 'awt' sink (stream query)")
                self.lhawt_sink.stop()
            except Exception as e:
                pass
            finally:
                self.lhawt_sink = None
                self.debug(f"`-> done!")
        if self.lhawt_console_sink is  not None:
            try:
                self.debug(f"TSP:stopping 'awt' console sink (stream query)")
                self.lhawt_console_sink.stop()
            except Exception as e:
                pass
            finally:
                self.lhawt_console_sink = None
                self.debug(f"`-> done!")
   
    # -------------------------------------------------------------------------------
    def stop_trains_progress_stream(self):
    # -------------------------------------------------------------------------------
        # stop the streaming queries (best effort impl.)
        if self.trprg_sink is  not None:
            try:
                self.debug(f"TSP:stopping trains progression sink (stream query)")
                self.trprg_sink.stop()
            except Exception as e:
                pass
            finally:
                self.trprg_sink = None
                self.debug(f"`-> done!")
                       
    # -------------------------------------------------------------------------------
    def cleanup(self):
    # -------------------------------------------------------------------------------
        # cleanup the underlying session 
        # TODO: not sure this is the right way to do the job
        self.stop()
        self.debug(f"TSP:shutting down Kafka-SparkSession")
        self.kafka_session.stop()
        self.debug(f"`-> done!")
      
    # -------------------------------------------------------------------------------
    def turnVerboseOn(self):
    # -------------------------------------------------------------------------------
        # turn verbose on          
        self.set_logging_level(logging.DEBUG)
       
    # -------------------------------------------------------------------------------
    def turnVerboseOff(self):
    # -------------------------------------------------------------------------------
        # turn verbose off   
        self.set_logging_level(logging.ERROR)
                       
    # -------------------------------------------------------------------------------
    def clearOutputs(self):
    # -------------------------------------------------------------------------------
        # clear outputs (i.e. clear our 'mother notebook-cell')
        clear_outputs_period = self.config.get('clear_outputs_period', 15)
        if (time.time() - self.last_clear_outputs_ts) > clear_outputs_period:
            self.clear_output()
            self.last_clear_outputs_ts = time.time()
   
    # -------------------------------------------------------------------------------
    def showTrainsProgressionTable(self):
    # -------------------------------------------------------------------------------
        # trains progression table will be diplayed        
        self.show_trprg_table = True
       
    # -------------------------------------------------------------------------------
    def hideTrainsProgressionTable(self):
    # -------------------------------------------------------------------------------
        # trains progression table will NOT be diplayed 
        self.show_trprg_table = False
                       
    # -------------------------------------------------------------------------------
    def computeAwtMetricsAndSaveAsTempViews(self, batch, batch_number):
    # -------------------------------------------------------------------------------
        # PART-I: COMPUTE AVERAGE WAITING TIME METRICS
        # --------------------------------------------
        # this 'forEachBatch' callback is attached to the our 'lhawt_sink' (streaming query)
        try:
            # clear cell content so that we don't cumulate the log 
            self.clearOutputs()
                              
            # be sure we have some data to handle (incoming dataframe not empty)
            # this will avoid creating empty tables on Hive side 
            if batch.rdd.isEmpty():
                self.warning(f"TSP:computeAwtMetrics: ignoring empty batch #{batch_number}")
                return

            self.debug(f"TSP:entering computeAwtMetrics for batch #{batch_number}...")
                              
            # PART-I: Q1.1 & Q1.3: ordered average waiting time in minutes (over last hour)
            self.debug(f"computing ordered average waiting time...")
            t = time.time()
            tmp = batch.orderBy(sf.asc("awt")).select(batch.station, batch.awt)    
            self.kafka_session.createDataFrame(tmp.rdd).createOrReplaceTempView("ordered_awt")
            self.debug(f"`-> took {round(time.time() - t, 2)} s")
                                                                  
            # PART-I: Q1.2: global average waiting time in minutes (over last hour)
            self.debug(f"computing global average waiting time...")
            t = time.time()
            tmp = batch.agg(sf.count("station").alias("number_of_stations"), 
                            sf.avg("awt").alias("global_awt"))
            self.kafka_session.createDataFrame(tmp.rdd).createOrReplaceTempView("global_awt")
            self.debug(f"`-> took {round(time.time() - t, 2)} s")
            
            # PART-I: Q1.4: min average waiting time in minutes (over last hour)
            self.debug(f"computing min. average waiting time...")
            t = time.time()
            tmp = batch.orderBy(sf.asc("awt")).limit(1).select(batch.station, 
                                                               batch.awt.alias("min_awt"))
            self.kafka_session.createDataFrame(tmp.rdd).createOrReplaceTempView("min_awt")
            self.debug(f"`-> took {round(time.time() - t, 2)} s")
           
            # PART-I: Q1.5: max average waiting time in minutes (over last hour)
            self.debug(f"computing min. average waiting time...")
            t = time.time()
            tmp = batch.orderBy(sf.desc("awt")).limit(1).select(batch.station, 
                                                                batch.awt.alias("max_awt"))
            self.kafka_session.createDataFrame(tmp.rdd).createOrReplaceTempView("max_awt")
            self.debug(f"`-> took {round(time.time() - t, 2)} s")
                              
            self.debug(f"TSP:computeAwtMetrics successfully executed for batch #{batch_number}")
        except Exception as e:
            self.error(f"TSP:failed to compute awt metrics from batch #{batch_number}")
            self.error(e)                            
                       
    # -------------------------------------------------------------------------------    
    def computeTrainsProgressionAndSaveAsTempView(self, batch, batch_number):
    # -------------------------------------------------------------------------------
        # PART-II: COMPUTE TRAINS PROGRESSION
        # ------------------------------------
        # this 'forEachBatch' callback is attached to the our 'trprg_sink' (streaming query)
        try:    
            # clear cell content so that we don't cumulate the log               
            # self.clear_output()
                              
            # be sure we have some data to handle (incoming dataframe not empty)
            # this will avoid creating empty tables on Hive side 
            if batch.rdd.isEmpty():
                self.warning(f"TSP:computeTrainsProgression ignoring empty batch #{batch_number}")
                return

            t = time.time()

            # create next_departure 'lead' columns: departure columns up shifted by 1 row
            tmp = batch.withColumn('next_departure', sf.lead('departure') \
                       .over(spark_window.partitionBy("train") \
                       .orderBy("departure")))
            # create next_station 'lead' columns: station columns up shifted by 1 row
            tmp = tmp.withColumn('next_station', sf.lead('station') \
                      .over(spark_window.partitionBy("train") \
                      .orderBy("departure")))
            # create humanly readable columns for departure date/time 
            tmp = tmp.withColumn("departure_date", sf.from_unixtime(tmp.departure, "HH:mm:ss"))
            tmp = tmp.withColumn("next_departure_date", sf.from_unixtime(tmp.next_departure, "HH:mm:ss"))

            # compute travel time between 'departure' and 'next_departure' - i.e. from one station to the next
            tmp = tmp.withColumn("time_to_st", tmp.next_departure -  tmp.departure)
                       
            # travel direction encoding: 1:paris->banlieue or -1:banlieue->paris
            tmp = tmp.withColumn("direction", 
                                 sf.when(tmp.mission.isin(self.config['missions_to_paris']), sf.lit(1)) \
                                   .otherwise(sf.lit(-1)))
                       
            # swap departure date/time (due to train direction) - this is just for readability & display 
            tmp = tmp.withColumn("temp_departure_date", tmp.departure_date)
            tmp = tmp.withColumn("departure_date", 
                                 sf.when(tmp.departure < tmp.next_departure, tmp.departure_date) \
                                   .otherwise(tmp.next_departure_date))
            tmp = tmp.withColumn("next_departure_date", 
                                 sf.when(tmp.departure < tmp.next_departure, tmp.next_departure_date) \
                                   .otherwise(tmp.temp_departure_date))
            tmp = tmp.drop("temp_departure_date")

            # tmp.show()

            # create column to store the current time (i.e. now)
            tmp = tmp.withColumn("now", sf.unix_timestamp(sf.current_timestamp()))

            # the travel (from one station to the next) can belong to the past, the future or can be in progress 
            tmp = tmp.withColumn("in_past", (tmp.now > tmp.departure) & (tmp.now > tmp.next_departure))
            tmp = tmp.withColumn("in_future", (tmp.now < tmp.departure) & (tmp.now < tmp.next_departure))
            tmp = tmp.withColumn("in_progress", (tmp.in_past != sf.lit(True)) & (tmp.in_future != sf.lit(True)))

            # tmp.show()

            # keep only 'in progress' travels - i.e. the ones not in past nor in the future
            # we also remove: 
            #    - rows for which next_departure is null (introduced by the lead function)
            # note that we keep:
            #    - trains in standby (i.e fake travel from one station to the same - train waiting for next departure)
            tmp = tmp.filter((~tmp.in_past & ~tmp.in_future) & (tmp.next_departure.isNotNull()))

            # tmp.show()

            # compute travel progression in %
            tmp = tmp.withColumn("progress", (100. * sf.abs((tmp.now - tmp.departure))) / sf.abs(tmp.time_to_st))  
            # compute trains progression: maintain value in  the [O, 100]% range 
            tmp = tmp.withColumn("progress", sf.when(tmp.progress < sf.lit(0.), sf.lit(0.)).otherwise(tmp.progress))             
            # compute trains progression: maintain value in  the [O, 100]% range 
            tmp = tmp.withColumn("progress", sf.when(tmp.progress > sf.lit(100.), sf.lit(100.)).otherwise(tmp.progress))

            # tmp.show()
              
            # select the required columns
            tmp = tmp.select(tmp.train, 
                             tmp.departure_date.alias("departure"),
                             tmp.next_departure_date.alias("arrival"),
                             tmp.mission, 
                             tmp.station.alias("from_st"), 
                             tmp.next_station.alias("to_st"), 
                             tmp.time_to_st,
                             tmp.progress,
                             tmp.direction) 
    
            # from (departure location)
            tmp = tmp.join(self.stations_data, tmp.from_st == self.stations_data.station, how="left")
            tmp = tmp.withColumn("from_st_lt", tmp.latitude).drop("latitude")
            tmp = tmp.withColumn("from_st_lg", tmp.longitude).drop("longitude")
            tmp = tmp.withColumn("from_st_lb", tmp.label).drop("label")
            tmp = tmp.drop("station")

            # to (destination location) 
            tmp = tmp.join(self.stations_data, tmp.to_st == self.stations_data.station, how="left")
            tmp = tmp.withColumn("to_st_lt", tmp.latitude).drop("latitude")
            tmp = tmp.withColumn("to_st_lg", tmp.longitude).drop("longitude")
            tmp = tmp.withColumn("to_st_lb", tmp.label).drop("label")
            tmp = tmp.drop("station")

            # compute current train latitude & longitude
            tmp = tmp.withColumn("train_lt", 
                                 tmp.from_st_lt + ((tmp.progress / 100.) * (tmp.to_st_lt - tmp.from_st_lt)))
            tmp = tmp.withColumn("train_lg", 
                                 tmp.from_st_lg + ((tmp.progress / 100.) * (tmp.to_st_lg - tmp.from_st_lg)))
              
            # cast progress to int
            tmp = tmp.withColumn("progress", sf.format_number(tmp.progress, 1).cast("double"))
            
            # compute accurate latitute and longitude using our user defined functions (udf)
            if self.accurate_trains_position_enabled:
                tmp = tmp.withColumn("train_alt", 
                                     TransilienStreamProcessor.alt_udf(tmp.from_st, tmp.to_st, tmp.progress))
                tmp = tmp.withColumn("train_alg", 
                                     TransilienStreamProcessor.alg_udf(tmp.from_st, tmp.to_st, tmp.progress))
            else:
                tmp = tmp.withColumn("train_alt", tmp.train_lt)
                tmp = tmp.withColumn("train_alg", tmp.train_lg)
                       
            # remove tmp data from table
            tmp = tmp.select("train",       # train identifier 
                             "departure",   # departure time
                             "arrival",     # arrival time
                             "mission",     # mission code
                             "from_st",     # departure station code
                             "to_st",       # arrival station code
                             "from_st_lb",  # departure station label
                             "to_st_lb",    # arrival station label
                             "time_to_st",  # time_to_st = arrival - departure in seconds
                             "progress",    # travel progress
                             "direction",   # 1: paris -> banlieue, -1: banlieue->paris
                             "train_lt",    # current train location: latitude 
                             "train_lg",    # current train location: longitude
                             "train_alt",   # current train location: latitude 
                             "train_alg")   # current train location: longitude
            
            # log/debug/validate...
            if self.show_trprg_table:
                tmp.show()
                       
            # create a temp. view that - visible from Tableau             
            self.kafka_session.createDataFrame(tmp.rdd).createOrReplaceTempView("trains_progression")
            self.debug(f"`-> took {round(time.time() - t, 2)} s")
                              
            self.debug(f"TSP:computeTrainsProgression successfully executed for batch #{batch_number}")
        except Exception as e:
            self.error(f"TSP:failed to compute trains progression from batch #{batch_number}")
            self.error(e)

Configuration de l'instance de la classe TransilienStreamProcessor:

In [6]:
# configuration parameters
config = {}

# json schema & options for kafka messages deserialization 
config['json_schema'] = st.StructType(
    [
        st.StructField("station", st.IntegerType(), True),
        st.StructField("train", st.StringType(), True),
        st.StructField("timestamp", st.TimestampType(), True),
        st.StructField("mode", st.StringType(), True),
        st.StructField("mission", st.StringType(), True),
        st.StructField("terminus", st.IntegerType(), True)
    ]
)
config['json_options'] = {"timestampFormat": "yyyy-MM-dd'T'HH:mm:ss.sss'Z'"}

# spark sesssions options
config['spark_sql_shuffle_partitions'] = 4

# kafka source configuration: broker & topic
config['kafka_broker'] = "sandbox-hdp.hortonworks.com:6667"
config['kafka_topic'] = "transilien-02"

# kafka stream configuration: 
# structured stream windowing for the last hour average waiting time stream
config['kafka_lhawt_stream_watermark'] = 1 
config['kafka_lhawt_stream_window_length'] = 60
config['kafka_lhawt_stream_sliding_interval'] = 2
config['kafka_lhawt_processing_time'] = 5
config['kafka_lhawt_console_sink_enabled'] = True 

# kafka stream configuration: 
# pseudo time window for the trains progression stream (logical batch window)
config['kafka_trprg_time_window'] = 3600

# local thrift server configuration
config['hive_thrift_server_port'] = 10015

# list of missions to Paris (for trains direction identification)
config['missions_to_paris'] = [
    "PALS", "PASA", "PEBU", 
    "PEGE", "POPI", "POPU", 
    "POSA", "POVA", "POPE"
]

# misc. options
config['auto_start'] = True
config['verbose'] = False

Instanciation du TransilienStreamProcessor...

Pour mémoire, voici une liste des actions possibles sur l'instance TransilienStreamProcessor:
- tsp.enable_accurate_trains_position() : enable accurate trains position comptuation
- tsp.disable_accurate_trains_position() : disable accurate trains position comptuation
- tsp.showTrainsProgressionTable() : display trains progression dataframe each time it's updated
- tsp.hideTrainsProgressionTable() : don't show trains progression dataframe
- tsp.turnVerboseOff() : switch logging level to logging.ERROR
- tsp.turnVerboseOn()  : switch logging level to logging.DEBUG
- tsp.stop()  : stop the streaming queries
- tsp.start() : (re)start the streaming queries

In [7]:
tsp = TransilienStreamProcessor(config)

In [ ]:
tsp.turnVerboseOn()

In [ ]:
tsp.turnVerboseOff()

**NB: laissons le _TransilienStreamProcessor_ tourner - si nécessaire nous pourrons l'arrêter avec la commande suivante:**

In [12]:
tsp.stop()

Affichage du contenu de la table **trains_progression** (produite dans _TransilienStreamProcessor.computeTrainsProgressionAndSaveAsTempView_)

In [ ]:
tsp.kafka_session.sql("select * from trains_progression").toPandas()

----
### Bokeh (& Google Maps) comme alternative à Tableau Desktop pour la visualisation dynamique des données 
---

L'animation suivante illustre ce qu'il est possible d'obtenir en comninant les fonctionnalités de notre classe **Task** à celle de la [la librairie Bokeh](https://bokeh.pydata.org/en/latest/). L'idée est d'implémenter une visualisation dynamique de la localisation des trains dans la cellule d'un notebook Jupyter.  

La classe **TrainsTracker** implémente la fonctionnalité proposée. 

Note: L'implémentation repose - à travers Bokeh - sur l'API Google Maps. L'utilisation de cette API exige une clé qui peut être obtenue - moyennant l'enregistrement d'un moyen de paiement - sur [Google Cloud](https://developers.google.com/maps/documentation/javascript/get-api-key).

![GitHub Logo](./trains-tracker.gif)

In [8]:
import json
import numpy as np
from scipy.interpolate import interp1d
from bokeh.io import output_notebook, push_notebook, show
from bokeh.models import GMapOptions, ColumnDataSource, LabelSet, Label
from bokeh.models import Circle, Diamond
from bokeh.models import Range1d, HoverTool, PanTool, WheelZoomTool, CrosshairTool
from bokeh.plotting import gmap
from tools.task import Task
from tools.logging import NotebookCellContent

In [15]:
class TrainsTracker(Task, NotebookCellContent):

    # -------------------------------------------------------------------------------
    def __init__(self, config):
    # -------------------------------------------------------------------------------
        Task.__init__(self, "TrainsTracker")
        NotebookCellContent.__init__(self, "TrainsTracker")
        output_notebook()
        self.config = config
        self.trains_to_paris = None
        self.trains_to_suburbs = None
        self.set_logging_level(self.config.get('loging_level', logging.ERROR))
        # accurate trains position: load paths from station to station
        self.accurate_trains_position_enabled = True
        
    # -------------------------------------------------------------------------------
    def on_init(self):
    # -------------------------------------------------------------------------------
        try:
            self.plot_handle = self.__setup_gmap()
            self.handle_periodic_message()
            p = self.config.get('update_period_in_seconds', 1.)
            self.enable_periodic_message(p)
        except Exception as e:
            self.error(e)

    # -------------------------------------------------------------------------------
    def on_exit(self):
    # -------------------------------------------------------------------------------
        try:
            self.clear_output()
        except Exception as e:
            self.error(e)

    # -------------------------------------------------------------------------------
    def handle_periodic_message(self):
    # -------------------------------------------------------------------------------
        try:
            self.__update_trains_position(self.trains_to_paris, 1)
            self.__update_trains_position(self.trains_to_suburbs, -1)
            push_notebook(handle=self.plot_handle)
        except Exception as e:
            self.error(e)
        
    # -------------------------------------------------------------------------------
    def __setup_gmap(self):
    # -------------------------------------------------------------------------------
        # GMAP SETUP -------- 
        map_options = GMapOptions(lat=48.885, lng=2.19, 
                                  map_type="roadmap", styles=map_style, zoom=12)
        p = gmap(self.config['gmap_api_key'], map_options=map_options)
        p.title.text = "Transilien Ligne L"
        p.plot_height=600
        p.plot_width=1200
        p.toolbar.logo = None
        p.outline_line_color = None
        p.grid.grid_line_color = None
        # TRAINS SETUP ------
        self.trains_to_paris = self.__setup_trains_components(p, "yellow")
        self.trains_to_suburbs = self.__setup_trains_components(p, "blue")
        # STATIONS SETUP ----
        # get stations data from our deticate spark table
        self.__setup_stations_components(p)
        # return notebook handle
        return show(p, notebook_handle=True)

    # -------------------------------------------------------------------------------
    def __setup_trains_components(self, plot, glyph_color):
    # -------------------------------------------------------------------------------
        source = ColumnDataSource(data=dict(lat=[], lon=[], tid=[], mission=[], departure=[], 
                                            arrival=[], from_st_lb=[], to_st_lb=[]))
        glyph = Diamond(x="lon", y="lat", size=17, fill_color=glyph_color, fill_alpha=0.4)
        renderer = plot.add_glyph(source_or_glyph=source, glyph=glyph)
        labels = LabelSet(x='lon', y='lat', text='tid', level='glyph',
                          x_offset=0, y_offset=5, text_align='center', 
                          text_font_size="9pt", source=source, render_mode='canvas')
        plot.add_layout(labels)
        tooltips = [
            ("Train", "@tid"),
            ("Mission", "@mission"),
            ("Départ", "@departure"),
            ("Gare Départ", "@from_st_lb"),
            ("Arrivée", "@arrival"),
            ("Gare Arrivée", "@to_st_lb")
        ]
        plot.add_tools(HoverTool(renderers=[renderer], tooltips=tooltips))
        return source 
    
    # -------------------------------------------------------------------------------
    def __setup_stations_components(self, plot):
    # -------------------------------------------------------------------------------
        # get stations data from our deticate spark table
        ks = self.config['transilien_stream_processor'].kafka_session
        stations = ks.sql("select * from stations_data").toPandas()
        # configure teh stations glyph (rendering on map)
        short_labels = []
        for label in stations['label']:
            if len(label) > 16:
                short_labels.append("".join([label[:12],"..."]))
            else:
                short_labels.append(label)
        stations_source = ColumnDataSource(data=dict(lat=stations.latitude, 
                                                     lon=stations.longitude, 
                                                     sid=stations.station, 
                                                     flb=stations.label, 
                                                     slb=short_labels))
        stations_glyph = Circle(x="lon", y="lat", size=6, fill_color="red", fill_alpha=0.8)
        stations_renderer = plot.add_glyph(source_or_glyph=stations_source, glyph=stations_glyph)
        stations_labels = LabelSet(x='lon', y='lat', text='slb', level='glyph',
                                   x_offset=0, y_offset=-15, text_align='center', 
                                   text_font_size="7pt", source=stations_source, 
                                   render_mode='canvas')
        plot.add_layout(stations_labels)
        tooltips_schema = [
            ("Gare", "@sid"),
            ("Label", "@flb")
        ]
        plot.add_tools(HoverTool(renderers=[stations_renderer], tooltips=tooltips_schema))
        
    # -------------------------------------------------------------------------------
    def enable_accurate_trains_position(self):
    # -------------------------------------------------------------------------------
         self.accurate_trains_position_enabled = True
    
    # -------------------------------------------------------------------------------
    def disable_accurate_trains_position(self):
    # -------------------------------------------------------------------------------
         self.accurate_trains_position_enabled = False
        
    # -------------------------------------------------------------------------------
    def __update_trains_position(self, data_source, direction):
    # -------------------------------------------------------------------------------
        # get a ref. to the kafka session
        ks = self.config['transilien_stream_processor'].kafka_session
        # get data from 'trains_progression' temp. view
        pdf = ks.sql(f"select * from trains_progression where direction == {direction}").toPandas()
        # select 'accurate' or 'standard' (direct trajectory between depature and arrival)
        lat_data = pdf.train_alt if self.accurate_trains_position_enabled else pdf.train_lt
        lon_data = pdf.train_alg if self.accurate_trains_position_enabled else pdf.train_lg
        # create a bokeh.ColumnDataSource for updated data
        new_data = ColumnDataSource(
                       data = dict(
                            lat=lat_data, 
                            lon=lon_data,
                            tid=pdf.train,
                            mission=pdf.mission,
                            departure=pdf.departure,
                            arrival=pdf.arrival,
                            from_st_lb=pdf.from_st_lb,
                            to_st_lb=pdf.to_st_lb
                       )
                   )
        # tell bokeh we changed the data source
        data_source.data.update(new_data.data)

In [16]:
# a free gmap style from https://snazzymaps.com
map_style = """ 
[
    {
        "featureType": "all",
        "elementType": "labels.text",
        "stylers": [
            {
                "visibility": "off"
            }
        ]
    },
    {
        "featureType": "road",
        "elementType": "all",
        "stylers": [
            {
                "visibility": "simplified"
            }
        ]
    },
    {
        "featureType": "road",
        "elementType": "geometry.stroke",
        "stylers": [
            {
                "visibility": "simplified"
            }
        ]
    },
    {
        "featureType": "road",
        "elementType": "labels.text",
        "stylers": [
            {
                "visibility": "off"
            }
        ]
    },
    {
        "featureType": "road",
        "elementType": "labels.icon",
        "stylers": [
            {
                "visibility": "off"
            }
        ]
    }
]
"""

Création du fichier  'gmap_api_key.json' contenant la clé Google Maps. Le fichier doit respecter le schéma json suivant: ("xxxxxxxxxxxxxxxx" étant la clé [Google Map API Key](https://developers.google.com/maps/documentation/javascript/get-api-key)): 
```
credentials = {'gmap_api_key': "xxxxxxxxxxxxxxxx"}
with open('./gmap_api_key.json', 'w+', encoding='utf-8') as f:
    json.dump(credentials, f)
```

Chargement de la clé depuis le fichier 'gmap_api_key.json' 

In [17]:
with open('./gmap_api_key.json', 'r', encoding='utf-8') as f:
    credentials = json.load(f)

Configuration du TrainsTracker

In [18]:
mtt_config = {
    'gmap_api_key': credentials['gmap_api_key'],
    'gmap_style': map_style,
    'update_period_in_seconds': 1.,
    'transilien_stream_processor': tsp
} 

Instanciation du TrainsTracker

In [19]:
mtt = TrainsTracker(mtt_config)
mtt.start_asynchronously()

In [24]:
mtt.disable_accurate_trains_position()

In [25]:
mtt.enable_accurate_trains_position()

Arrêt (et destruction) de notre TrainsTracker.

Note: Un appel à _exit_ demande au Thread sous-jacent de retourner - ce qui provoque sa 'disparition'. Il existe aucun moyen de redémarrer une Task sur laquelle _exit_ a été invoquée. L'objet doit être reconstruit. 

In [26]:
mtt.exit()